<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/didactica_ciencia_datos/blob/main/referentes/geron/part_2/c_10/c_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# Capítulo 11 

# Chapter 11. Entrenamiento de redes neuronales profundas

En el Capítulo 10 presentamos las redes neuronales artificiales y entrenamos nuestras primeras redes neuronales profundas.

Pero eran redes poco profundas, con solo unas pocas capas ocultas.

¿Qué sucede si necesita abordar un problema complejo, como detectar cientos de tipos de objetos en imágenes de alta resolución?

Es posible que deba entrenar un DNN mucho más profundo, tal vez con 10 capas o muchas más, cada una con cientos de neuronas, unidas por cientos de miles de conexiones.

Entrenar un DNN profundo no es un paseo por el parque. 

Estos son algunos de los problemas con los que podría encontrarse:

Es posible que se enfrente al complicado problema de los gradientes que se desvanecen o al problema relacionado con los gradientes explosivos.

Esto es cuando los gradientes se vuelven más y más pequeños, o más y más grandes, cuando fluyen hacia atrás a través de la DNN durante el entrenamiento.

Ambos problemas hacen que las capas inferiores sean muy difíciles de entrenar.

Es posible que no tenga suficientes datos de entrenamiento para una red tan grande o que sea demasiado costoso etiquetarlos.

El entrenamiento puede ser extremadamente lento.

Un modelo con millones de parámetros correría el grave riesgo de sobreajustar el conjunto de entrenamiento, especialmente si no hay suficientes instancias de entrenamiento o si son demasiado ruidosas.

En este capítulo repasaremos cada uno de estos problemas y presentaremos técnicas para resolverlos.

Comenzaremos explorando los problemas de gradientes que se desvanecen y explotan y algunas de sus soluciones más populares.

A continuación, veremos el aprendizaje por transferencia y el entrenamiento previo no supervisado, que pueden ayudarlo a abordar tareas complejas incluso cuando tiene pocos datos etiquetados.

Luego discutiremos varios optimizadores que pueden acelerar enormemente el entrenamiento de modelos grandes.

Finalmente, veremos algunas técnicas de regularización populares para redes neuronales grandes.

Con estas herramientas podrás entrenar redes muy profundas.

¡Bienvenido al Aprendizaje Profundo!

## Los problemas de gradientes de desaparición/explosión

Como discutimos en el Capítulo 10, el algoritmo de retropropagación funciona yendo de la capa de salida a la capa de entrada, propagando el gradiente de error a lo largo del camino.

Una vez que el algoritmo ha calculado el gradiente de la función de costo con respecto a cada parámetro en la red, usa estos gradientes para actualizar cada parámetro con un paso de descenso de gradiente.

Unfortunately, gradients often get smaller and smaller as the algorithm progresses down to the lower layers. 

As a result, the Gradient Descent update leaves the lower layers’ connection weights virtually unchanged, and training never converges to a good solution. 

We call this the vanishing gradients problem. In some cases, the opposite can happen: the gradients can grow bigger and bigger until layers get insanely large weight updates and the algorithm diverges. 

This is the exploding gradients problem, which surfaces in recurrent neural networks (see Chapter 15). 

More generally, deep neural networks suffer from unstable gradients; different layers may learn at widely different speeds.

This unfortunate behavior was empirically observed long ago, and it was one of the reasons deep neural networks were mostly abandoned in the early 2000s. 

It wasn’t clear what caused the gradients to be so unstable when training a DNN, but some light was shed in a 2010 paper by Xavier Glorot and Yoshua Bengio. 

The authors found a few suspects, including the combination of the popular logistic sigmoid activation function and the
weight initialization technique that was most popular at the time (i.e., a normal distribution with a mean of 0 and a standard deviation of 1). 

In short, they showed that with this activation function and this initialization scheme, the variance of the outputs of each layer is much greater than the variance of its inputs. 

Going forward in the network, the variance keeps increasing after each layer until the activation function saturates at the top
layers. 

This saturation is actually made worse by the fact that the logistic function has a mean of 0.5, not 0 (the hyperbolic tangent function has a mean of 0 and behaves slightly better than the logistic function in deep networks).

Looking at the logistic activation function (see Figure 11-1), you can see that when inputs become large (negative or positive), the function saturates at 0 or 1, with a derivative extremely close to 0. 

Thus, when backpropagation kicks in it has virtually no gradient to propagate back through the network; and what little gradient exists keeps getting diluted as backpropagation progresses down through the top layers, so there is really nothing left for the lower layers.

<img src = 'https://github.com/marco-canas/didactica_ciencia_datos/blob/main/referentes/geron/part_2/c_11/figure_11_1.jpg?raw=true'>

## Glorot and He Initialization


In their paper, Glorot and Bengio propose a way to significantly alleviate the unstable gradients problem. 

They point out that we need the signal to flow properly in both directions: in the forward direction when making predictions, and in the reverse direction when backpropagating gradients.

We don’t want the signal to die out, nor do we want it to explode and saturate. 

For the signal to flow properly, the authors argue that we need the variance of the outputs of each layer to be equal to the variance of its inputs, and we need the gradients to have equal variance before and after flowing through a layer in the reverse direction (please check out the paper if you are interested in the mathematical details). 

It is actually not possible to guarantee both unless the layer has an equal number of inputs and neurons (these numbers are called the fan-in and fan-out of the layer), but Glorot and Bengio proposed a good compromise that has proven to work very well in practice: the connection weights of each layer must be initialized randomly as described in Equation 11-1, where 
$fanavg = (fanin + fanout)/2$. 

This initialization strategy is called Xavier initialization or Glorot initialization, after the paper’s first author.
Equation 11-1. Glorot initialization (when using the logistic activation function)

$$ \text{Normal distribucion with mean 0 and variance } \ \sigma^{2} = \frac{1}{fac_{avg}} $$